<img align="right" width="400" src="https://www.fhnw.ch/de/++theme++web16theme/assets/media/img/fachhochschule-nordwestschweiz-fhnw-logo.svg" alt="FHNW Logo">


# N-gram Language Model

by Fabian Märki

## Summary
The aim of this notebook is to build a n-gram Language Model and generate text using this model. Please note: Nowadays there are more efficient methods available to build a Language Model. This notebook is meant for for illustrative purposes only.

## Links
- [N-gram Language Models](https://nlpforhackers.io/language-models)
- [NLTK Language Modeling Module](https://kite.com/python/docs/nltk.lm)
- [Kaggle on N-gram Language Model with NLTK](https://www.kaggle.com/alvations/n-gram-language-model-with-nltk)

This notebook contains assigments: <font color='red'>Questions are written in red.</font>

<a href="https://colab.research.google.com/github/markif/2024_HS_DAS_NLP_Notebooks/blob/master/05_a_N-gram_Language_Model.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture

import nltk
import numpy as np

from nltk.corpus import reuters
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader
from nltk import bigrams, trigrams
from nltk.lm.preprocessing import pad_both_ends

from collections import Counter
from collections import Counter, defaultdict

Let's build a LM on the Reuter corpus.

In [2]:
%%capture

nltk.download('reuters')
nltk.download('punkt')

In [3]:
counts = Counter(reuters.words())

In [4]:
print("Total number of words:", len(reuters.words()))
print("Total number of unique words:", len(counts.keys()))

Total number of words: 1720901
Total number of unique words: 41600


NLTK offers bigram and trigram functionalities...

In [5]:
start_symbol = '<s>'
end_symbol = '</s>'
start_end_symbols = [start_symbol, end_symbol]
n_gram = 3

first_sentence = reuters.sents()[0]

In [6]:
# Build bigrams
print (list(bigrams(first_sentence)))

[('ASIAN', 'EXPORTERS'), ('EXPORTERS', 'FEAR'), ('FEAR', 'DAMAGE'), ('DAMAGE', 'FROM'), ('FROM', 'U'), ('U', '.'), ('.', 'S'), ('S', '.-'), ('.-', 'JAPAN'), ('JAPAN', 'RIFT'), ('RIFT', 'Mounting'), ('Mounting', 'trade'), ('trade', 'friction'), ('friction', 'between'), ('between', 'the'), ('the', 'U'), ('U', '.'), ('.', 'S'), ('S', '.'), ('.', 'And'), ('And', 'Japan'), ('Japan', 'has'), ('has', 'raised'), ('raised', 'fears'), ('fears', 'among'), ('among', 'many'), ('many', 'of'), ('of', 'Asia'), ('Asia', "'"), ("'", 's'), ('s', 'exporting'), ('exporting', 'nations'), ('nations', 'that'), ('that', 'the'), ('the', 'row'), ('row', 'could'), ('could', 'inflict'), ('inflict', 'far'), ('far', '-'), ('-', 'reaching'), ('reaching', 'economic'), ('economic', 'damage'), ('damage', ','), (',', 'businessmen'), ('businessmen', 'and'), ('and', 'officials'), ('officials', 'said'), ('said', '.')]


In [7]:
# Build trigrams
print (list(trigrams(first_sentence)))

[('ASIAN', 'EXPORTERS', 'FEAR'), ('EXPORTERS', 'FEAR', 'DAMAGE'), ('FEAR', 'DAMAGE', 'FROM'), ('DAMAGE', 'FROM', 'U'), ('FROM', 'U', '.'), ('U', '.', 'S'), ('.', 'S', '.-'), ('S', '.-', 'JAPAN'), ('.-', 'JAPAN', 'RIFT'), ('JAPAN', 'RIFT', 'Mounting'), ('RIFT', 'Mounting', 'trade'), ('Mounting', 'trade', 'friction'), ('trade', 'friction', 'between'), ('friction', 'between', 'the'), ('between', 'the', 'U'), ('the', 'U', '.'), ('U', '.', 'S'), ('.', 'S', '.'), ('S', '.', 'And'), ('.', 'And', 'Japan'), ('And', 'Japan', 'has'), ('Japan', 'has', 'raised'), ('has', 'raised', 'fears'), ('raised', 'fears', 'among'), ('fears', 'among', 'many'), ('among', 'many', 'of'), ('many', 'of', 'Asia'), ('of', 'Asia', "'"), ('Asia', "'", 's'), ("'", 's', 'exporting'), ('s', 'exporting', 'nations'), ('exporting', 'nations', 'that'), ('nations', 'that', 'the'), ('that', 'the', 'row'), ('the', 'row', 'could'), ('row', 'could', 'inflict'), ('could', 'inflict', 'far'), ('inflict', 'far', '-'), ('far', '-', 'rea

Now we can collect our counts in order to build a trigram model.

Let's build a (self made) trigram Model using the [pad_both_ends](https://www.nltk.org/api/nltk.lm.html#module-nltk.lm.preprocessing) and [trigrams](https://kite.com/python/docs/nltk.trigrams) function from NLTK.

In [8]:
%%time

trigramModel = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in reuters.sents():
    sentence = pad_both_ends(sentence, n=n_gram, left_pad_symbol=start_symbol, right_pad_symbol=end_symbol)
    for w1, w2, w3 in trigrams(sentence):
        trigramModel[(w1, w2)][w3] += 1

CPU times: user 7.19 s, sys: 245 ms, total: 7.43 s
Wall time: 7.45 s


Let us have some look on sample results

In [9]:
print(trigramModel["what", "the"]["economists"])
print(trigramModel["what", "the"]["nonexistingword"])
print(trigramModel[start_symbol, start_symbol]["The"])

2
0
8839


Let's calculate the probabilities 

In [10]:
for w1_w2 in trigramModel:
    total_count = sum(trigramModel[w1_w2].values())
    for w3 in trigramModel[w1_w2]:
        trigramModel[w1_w2][w3] /= total_count

In [11]:
print (trigramModel["what", "the"]["economists"])
print (trigramModel["what", "the"]["nonexistingword"] )
print (trigramModel[start_symbol, start_symbol]["The"] )

0.043478260869565216
0.0
0.16155800478879934


<font color='red'>**TASK: Play with the random parameter `r` in order to genereate more/less diverse/coherent texts.**</font>

In [12]:
import random
  
text = [start_symbol, start_symbol]
 
sentence_finished = False
 
while not sentence_finished:
    #random.seed(40)
    r = random.random()
    #r = random.uniform(0.0, 0.3)
    
    accumulator = .0
 
    for word in trigramModel[tuple(text[-(n_gram-1):])].keys():
        accumulator += trigramModel[tuple(text[-(n_gram-1):])][word]
         
        # bring in some randomness by not just taking the first word 
        # (the one with the highest probability) but sum the word's
        # probabilities until the sum is >= the randomly drawn value
        if accumulator >= r:
            text.append(word)
            break
 
    sentence_finished = text[-(n_gram-1):] == [end_symbol, end_symbol]
 
print(' '.join([t for t in text if t not in start_end_symbols]))

The department said .


Sounds almost like real text (at least sometimes)!

## 4-gram Language Model

Build a 4-gram Language Model by training a Maximum Likelihood Estimator [MLE](https://kite.com/python/docs/nltk.lm.MLE). You can get inspiration from [here](https://kite.com/python/docs/nltk.lm).

Alternatively to the Reuters corpus, you might want to consider training a model with an alternative corpus (e.g. Shakespeare from the [Gutenberg corpus](http://www.nltk.org/book_1ed/ch02.html)).

<strong>Note:</strong> For more sophisticated n-gram models, take a look at [these objects from nltk.lm.models](https://github.com/nltk/nltk/blob/develop/nltk/lm/models.py):

- Laplace: Implements Laplace (add one) smoothing.
- Lidstone: Provides Lidstone-smoothed scores.
- WittenBellInterpolated: Interpolated version of Witten-Bell smoothing.
- KneserNeyInterpolated: Interpolated version of Kneser-Ney smoothing.

In [13]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [14]:
n_gram = 4

Get the data, tokenize the sentences and build the training set.

In [15]:
# Tokenize the text.

tokenized_text = reuters.sents()

In [16]:
# Preprocess the tokenized text for n-grams language modelling

train_data, padded_sents = padded_everygram_pipeline(n_gram, tokenized_text)

### Train a n-gram Model

Having prepared our data we are ready to start training a model using an instance of Maximum Likelihood Estimator (MLE).

In [17]:
model = MLE(n_gram)

Train the model using its fit method.

In [18]:
%%time

model.fit(train_data, padded_sents)

CPU times: user 1min 21s, sys: 635 ms, total: 1min 22s
Wall time: 1min 22s


In [19]:
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 41602 items>


In some cases you might want to ignore words that did occur during training but did not occur frequently enough to provide useful information. You can tell the vocabulary to ignore such words using the unk_cutoff argument of [nltk.lm.vocabulary.Vocabulary](https://github.com/nltk/nltk/blob/develop/nltk/lm/vocabulary.py)

Let's test if an unknown word will be mapped to UNK.

In [20]:
print(model.vocab.lookup(tokenized_text[0]))
# If we lookup the vocab on unseen sentences not from the training data, 
# it automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('language is never unknownword .'.split()))

('ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.')
('language', 'is', 'never', '<UNK>', '.')


The model provides a convenient interface to access counts...

In [21]:
print(model.counts['businessmen']) # i.e. Count('businessmen')
print(model.counts[['businessmen']]['and']) # i.e. Count('and'|'businessmen')
print(model.counts[['businessmen', 'and']]['officials']) # i.e. Count('officials'|'businessmen and')
print(model.counts[["what", "the"]]['economists']) # i.e. Count('economists'|'what the')
print(model.counts[[start_symbol, start_symbol]]['The']) # i.e. Count('The'|'<s> <s>')

56
6
2
2
8839


The real purpose of training a language model is to have it score how probable words are in certain contexts...

In [22]:
print(model.score('businessmen')) # P('businessmen')
print(model.score('and', 'businessmen'.split()))  # P('and'|'businessmen')
print(model.score('officials', 'businessmen and'.split()) ) # P('officials'|'businessmen and')
print(model.score('economists', 'what the'.split()))  # P('economists'|'what the')
print(model.score('The', '<s> <s>'.split()) ) # P('The'|'<s> <s>')

2.732796436433447e-05
0.10714285714285714
0.3333333333333333
0.043478260869565216
0.08077900239439967


### Text Generation using n-gram Language Model


One feature of n-gram models is that they can be used to generate text.

In [23]:
print(model.generate(60, random_seed=42))

['feasibility', 'and', 'strategies', 'of', 'gaining', 'control', 'of', 'USAir', 'Group', "'", 's', 'acquisition', 'Portsmouth', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


Let's implement a generate_sent function that generates a sentence based on a n-gram language model. We use [TreebankWordDetokenizer](https://kite.com/python/docs/nltk.treebank.TreebankWordDetokenizer) and apply some additional cleaning to the generated tokens in order to make the generated sentence more human-like.

<font color='red'>**TASK: Extend `generate_sent` so that the generated text sequence conditions on an existing text sequence (so that the generated text starts with a predefined word sequence).**</font>

You might get inspiration from [here](https://www.nltk.org/api/nltk.lm.api.html#nltk.lm.api.LanguageModel.generate), i.e. the method `generate` has a parameter `text_seed` so that the generated text can be conditioned on preceding context.

In [24]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :model: A n-gram language model from `nltk.lm.model`.
    :num_words: Max no. of words to generate.
    :random_seed: Seed value for random.
    """
    content = []
    
    for token in model.generate(num_words, random_seed=random_seed):
        if token == start_symbol:
            continue
        if token == end_symbol:
            break
        content.append(token)
        
    return detokenize(content)

In [25]:
generate_sent(model, 60, random_seed=42)

"feasibility and strategies of gaining control of USAir Group' s acquisition Portsmouth."